<a href="https://colab.research.google.com/github/maguid28/EE6041-TAandNLP/blob/main/Etivity_6_1_23222425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Given the two confusion tables below, compute
Microaveraged precision, recall, and F1
Macroaveraged precision, recall, and F1
Explain the reason for the difference between the obtained Microaveraged and Macroaveraged F1 measures.



## a). Microaveraged precision, recall, and F1

$$ \text{Precision} = \frac{TP}{TP + FP} = \frac{870}{870 + 230} = \frac{870}{1100} = 0.791$$


--------------------------------------------------------------
$$ \text{Recall} = \frac{TP}{TP + FN} = \frac{870}{870 + 230} = 0.791 $$



--------------------------------------------------------------

$$ \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} =  \frac{2 \times \text{0.7909} \times \text{0.7909}}{\text{0.7909} + \text{0.7909}} = \frac{1.2510}{1.5818} = 0.791$$


## b). Macroaveraged precision, recall, and F1



Class: Food
  $$ \text{Precision} = \frac{TP}{TP + FP} = \frac{800}{800 + 200}  = \frac{800}{1000} = 0.8$$

Class: Drink
  $$ \text{Precision} = \frac{TP}{TP + FP} = \frac{70}{70 + 30}  = \frac{70}{100} = 0.7$$

Macro Avg:
  $$ \text{Macro average Precision} = \frac{0.8 + 0.7}{2} = 0.75$$


--------------------------------------------------------------

$$ \text{Recall} = \frac{TP}{TP + FN} $$

Class: Food
  $$ \text{Recall} = \frac{800}{800 + 200} = \frac{800}{1000} = 0.8$$

Class: Drink
  $$ \text{Recall} = \frac{70}{70 + 30} = \frac{70}{70 + 30} = 0.7$$

Macro Avg:
  $$ \text{Macro average Recall} = \frac{0.8 + 0.7}{2} = 0.75$$

--------------------------------------------------------------

$$ \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}  $$

  $$ \text{F1 Score} = \frac{2 \times \text{0.75} \times \text{0.75}}{\text{0.75} + \text{0.75}} =  \frac{1.125}{1.5} = 0.75$$


# C). Explain the reason for the difference between the obtained Microaveraged and Macroaveraged F1 measures
The difference in the micro and macro averaging is because for Macroaveraging we compute the each class individually and then average, whereas for microaveraging we take the two classes, add them together to get the micro averaged table.
Macroaveraging has the benefit of giving classes with less values a stronger representation.
In microaveraging if one classes values are too small then that classes impact on the final average can be minimal.

# Task 2: Modify the Multinomial Naïve Bayes classifier function that you developed in Etivity5, Task3 to train and test a sentiment classifier

In [ ]:
trainingSet = [('just plain boring','-'),('entirely predictable and lacks energy','-'),('no surprises and very few laughs','-'),('very powerful','+'),('the most fun film of the summer','+')]
testSet = [('predictable with no fun','?')]

In [ ]:
def calculate_prior_probabilities(dataset):
    total_samples = len(dataset)
    #print(f"total samples:  {total_samples}")
    #print(dataset)
    neg_count = 0
    pos_count = 0

    for _, label in dataset :
      if label == '-':
        neg_count+=1
      else :
        pos_count+=1

    #print(neg_count)
    #print(dataset)

    prior_probability_neg = neg_count / total_samples
    prior_probability_pos = pos_count / total_samples

    return prior_probability_neg, prior_probability_pos

In [ ]:
priorPOS, priorNEG = calculate_prior_probabilities(trainingSet)

In [ ]:
def create_bow(dataset, min_frequency = 1):
    all_words = []
    for sample, _ in dataset:
        all_words.extend(sample.lower().split())

    word_frequencies = {}
    for word in all_words:
        if word in word_frequencies:
            word_frequencies[word] += 1
        else:
            word_frequencies[word] = 1

    bag_of_words = {word: count for word, count in word_frequencies.items() if count >= min_frequency}
    return bag_of_words

In [ ]:
bag_of_words= create_bow(trainingSet)

In [ ]:
def calculate_likelihoods(dataset, bag_of_words):
    laplace_smoothing = 1
    pos_samples = [text.lower() for text, label in dataset if label == 1]
    neg_samples = [text.lower() for text, label in dataset if label == 0]

    pos_word_counts = {word: laplace_smoothing for word in bag_of_words}
    neg_word_counts = {word: laplace_smoothing for word in bag_of_words}

    for text in pos_samples:
        for word in text.split():
            if word in pos_word_counts:
                pos_word_counts[word] += 1

    for text in neg_samples:
        for word in text.split():
            if word in neg_word_counts:
                neg_word_counts[word] += 1

    pos_total_count = sum(pos_word_counts.values())
    neg_total_count = sum(neg_word_counts.values())

    pos_likelihoods = {word: pos_word_counts[word] / pos_total_count for word in pos_word_counts}
    neg_likelihoods = {word: neg_word_counts[word] / neg_total_count for word in neg_word_counts}

    return pos_likelihoods, neg_likelihoods

In [ ]:
def posterior_probability(new_sample, prior_probabilities, likelihoods):
    neg_likelihoods, pos_likelihoods = likelihoods
    prior_probability_neg, prior_probability_pos = prior_probabilities

    words = new_sample.lower().split()

    log_pos_prob = np.log(prior_probability_pos)
    log_neg_prob = np.log(prior_probability_neg)

    for word in words:
        pos_likelihood = pos_likelihoods.get(word, 1 / (sum(pos_likelihoods.values()) + len(pos_likelihoods)))
        ie_likelihood = neg_likelihoods.get(word, 1 / (sum(neg_likelihoods.values()) + len(neg_likelihoods)))

        print(f"Word: {word} \t wordConditionalProbPos: {pos_likelihood} \t wordConditionalProbNeg: {ie_likelihood}")

        log_pos_prob += np.log(pos_likelihood)
        log_neg_prob += np.log(ie_likelihood)

    #Convert back to normal probabilities
    pos_prob = np.exp(log_pos_prob)
    neg_prob = np.exp(log_neg_prob)

    total_prob = pos_prob + neg_prob
    posterior_probability_pos = pos_prob / total_prob
    posterior_probability_neg = neg_prob / total_prob

    return posterior_probability_pos, posterior_probability_neg

In [ ]:
NEG_likelihoods, POS_likelihoods = calculate_likelihoods(trainingSet, bag_of_words)
likelihoods = (NEG_likelihoods, POS_likelihoods)

In [ ]:
def classify_text(sentence, prior_probabilities, likelihoods):
    posterior_probability_pos, posterior_probability_neg = posterior_probability(sentence, prior_probabilities, likelihoods)
    print(f"\ndocProbPos: {posterior_probability_pos}")
    print(f"docProbNeg: {posterior_probability_neg}")

    if posterior_probability_pos > posterior_probability_neg:
        classification = "Positive"
    elif posterior_probability_neg > posterior_probability_pos:
        classification = "Negative"
    else:
        classification = "Neutral"

    return classification

In [ ]:
import math
from collections import Counter
import numpy as np

def naiveBayesClassifier(trainingSet,testSet):
  priors = calculate_prior_probabilities(trainingSet)

  for doc in testSet:
    print("--------------------------------------------------------------------------------")
    print(f"\nTest Document: {doc} \n")
    classified_result = classify_text(doc[0], priors, likelihoods)
    print(f"\nInferred Class: {classified_result}")

naiveBayesClassifier(trainingSet,testSet)

--------------------------------------------------------------------------------

Test Document: ('predictable with no fun', '?') 

Word: predictable 	 wordConditionalProbPos: 0.05 	 wordConditionalProbNeg: 0.05
Word: with 	 wordConditionalProbPos: 0.047619047619047616 	 wordConditionalProbNeg: 0.047619047619047616
Word: no 	 wordConditionalProbPos: 0.05 	 wordConditionalProbNeg: 0.05
Word: fun 	 wordConditionalProbPos: 0.05 	 wordConditionalProbNeg: 0.05

docProbPos: 0.4000000000000002
docProbNeg: 0.5999999999999999

Inferred Class: Negative


# Task 3: Write a Sentiment Analysis function that takes a string as input and identifies its sentiment using the TextBlob library.

In [ ]:
from textblob import TextBlob

def sentimentAnalyzer(text):
    analysis = TextBlob(text)
    print("----------------------------------")
    print(f"String = {text}\n" )
    if analysis.sentiment[0] > 0.1 :
      print("Positive Sentiment ")
    elif analysis.sentiment[0] < -0.1 :
      print("Negative Sentiment ")
    else :
      print("Neutral Sentiment")

    print(analysis.sentiment)

In [ ]:
sentimentAnalyzer("NLP is cool")
sentimentAnalyzer("NLP is cool and useful")
sentimentAnalyzer("NLP is hard")
sentimentAnalyzer("NLP is hard and useless")
sentimentAnalyzer("NLP stands for Natural Language Processing")

----------------------------------
String = NLP is cool

Positive Sentiment 
Sentiment(polarity=0.35, subjectivity=0.65)
----------------------------------
String = NLP is cool and useful

Positive Sentiment 
Sentiment(polarity=0.32499999999999996, subjectivity=0.325)
----------------------------------
String = NLP is hard

Negative Sentiment 
Sentiment(polarity=-0.2916666666666667, subjectivity=0.5416666666666666)
----------------------------------
String = NLP is hard and useless

Negative Sentiment 
Sentiment(polarity=-0.39583333333333337, subjectivity=0.37083333333333335)
----------------------------------
String = NLP stands for Natural Language Processing

Neutral Sentiment
Sentiment(polarity=0.1, subjectivity=0.4)
